In [ ]:
import numpy as np
import control as ctrl
from sympy import symbols, Matrix
from lib.OrderUncertainties import *
from src.Benchmarks import Bench, sys_variables


In [ ]:
def get_sensitivity(A, B):
    mat = OrdUnc(A)
    if mat.determineCase() == 1:
        unc = mat.distinctPos(B)
    else:
        unc = mat.multSig(B)
    return unc

def all_sensitivity(bench, K):
    # A_bar_i = A + BKE_i where E_i = I(n) with the (i, i) entry swapped to lambda
    for i in range(bench.nx):
        E = Matrix(np.eye(bench.nx))
        E[i, i] = symbols('l')
        A = bench.A + bench.B * K * E
        return A

def find_AB(sys, x):
    nx = sys.A.shape[0]
    l = symbols('l')
    E = np.eye(nx)
    E[x,x] = l
    print(nx)

def main():
    pass

F1 = sys_variables['F1']
K = ctrl.lqr(F1.sysd, np.eye(F1.nx), np.eye(F1.nu))[0]
print(K)
# all_sensitivity(F1, K)

# main()
